In [ ]:
def nextWord(inp):
  x = []
  y = []
  for i in range(len(inp)-4):
    x.append(inp[i:i+4])
    y.append(inp[i+4])
  return x,y

#x,y = nextWord(text.split())

Cleaning

In [ ]:
import re
CLEANR = re.compile('<.*?>') 

In [ ]:
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [ ]:
train

Trying something

In [ ]:
def makeDataset(dt):
  x = []
  y = []
  for text in dt:
    seq,pred = nextWord(text.split())
    x += seq
    y += seq
  return x,y

Self supervised learning

In [ ]:
from nltk.corpus import stopwords
from collections import Counter
from nltk import *
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import numpy as np
import random
import pandas as pd
import sys
import os
import time
import codecs
import collections
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy
from scipy import spatial
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer = ToktokTokenizer()

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Input, LSTM, Embedding,Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Conv1D, SimpleRNN
from keras.models import Model
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras import initializers, regularizers, constraints,optimizers, layers
from keras.layers import Dense, Input, Flatten, Dropout,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential
from keras.layers.recurrent import SimpleRNN
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score



In [ ]:
from google.colab import files 
upload = files.upload()
!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 43% 11.0M/25.7M [00:00<00:00, 113MB/s]
100% 25.7M/25.7M [00:00<00:00, 126MB/s]


In [ ]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
df = pd.read_csv('IMDB Dataset.csv')
text = df.review.tolist()

In [ ]:
# Convert the given list to strings
from collections import Iterable

def reduce_dims(items):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in reduce_dims(x):
                yield sub_x
        else:
            yield x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [ ]:
string_final = ''.join(text)


This part of tokenization part needs to be changed.

In [ ]:
def preprocessString(s):
  s = s.replace('\n', '')
  s = s.lower()
  pattern = r'[^a-zA-z0-9\s]'
  s = re.sub(pattern, "", s)
  return s

In [ ]:
def tokeniz(s):
  tokens = tokenizer.tokenize(s)
  tokens = [token.strip() for token in tokens]
  return tokens  

In [ ]:
string_final = preprocessString(string_final)
tokens = tokeniz(string_final)
total_words = Counter(tokens)
vocab_len=len(total_words)

In [ ]:
words = [x[0] for x in total_words.most_common()]
len(words)
sorted_words = list(sorted(words))
sorted_words[:10]
word_ind = {x: i for i, x in enumerate(sorted_words)}



In [ ]:
inp = []
out = []
sentence_length =4
for i in range(0, len(total_words) - sentence_length, 1):
    x = tokens[i:i+sentence_length]
    y = tokens[i+sentence_length]
    # Creating a vector
    inp.append([word_ind[char] for char in x])
    out.append(word_ind[y])

In [ ]:
len(out)

208150

In [ ]:
train_Y = out[0:2000]
train_X = inp[0:2000]

# Inverse dictionary
inv_dict = dict(map(reversed, word_ind.items()))



In [ ]:
X = numpy.reshape(train_X, (len(train_X), sentence_length, 1))

In [ ]:
# to_categorical for one-hot encoding
Y = np_utils.to_categorical(train_Y)

In [ ]:
X.shape

(2000, 4, 1)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
# model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True))
# model.add(Dropout(0.2))
model.add(LSTM(256,))
# model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

file_name_path="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_name_path, monitor='loss',
verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]



In [ ]:
model.fit(X, Y, epochs=3, batch_size=32, callbacks=callbacks) 



Epoch 1/3
62/63 [============================>.] - ETA: 0s - loss: 9.6317
Epoch 00001: loss improved from inf to 9.63527, saving model to weights-improvement-01-9.6353.hdf5
63/63 [==============================] - 13s 89ms/step - loss: 9.6353
Epoch 2/3
62/63 [============================>.] - ETA: 0s - loss: 6.7199
Epoch 00002: loss improved from 9.63527 to 6.71886, saving model to weights-improvement-02-6.7189.hdf5
63/63 [==============================] - 5s 86ms/step - loss: 6.7189
Epoch 3/3
63/63 [==============================] - ETA: 0s - loss: 6.3169
Epoch 00003: loss improved from 6.71886 to 6.31686, saving model to weights-improvement-03-6.3169.hdf5
63/63 [==============================] - 8s 126ms/step - loss: 6.3169


Testing the model

In [ ]:
test_Y = out[2000:3000]
test_X = inp[2000:3000]

In [ ]:
y_pred=model.predict(
    test_X,
    batch_size=32,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

In [ ]:
y_pred=np.argmax(y_pred,axis=1).reshape((1000,1))

In [ ]:
def accuracy(y_pred,y):
    tot = len(y)
    corr = 0
    for i in range(len(y_pred)):
        corr += (y_pred[i] == y[i])
    return corr/tot

In [ ]:
accuracy(y_pred,test_Y)

array([0.059])

###Adapting to IMDB dataset for classification

```
##But how do we adapt the first layer which takes 4 words to take in a sentence

```



In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 256)            264192    
                                                                 
 lstm_1 (LSTM)               (None, 4, 256)            525312    
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 207908)            53432356  
                                                                 
Total params: 54,747,172
Trainable params: 54,747,172
Non-trainable params: 0
_________________________________________________________________


In [ ]:
clf = Sequential()
for layer in model.layers[:-1]: # go through until last layer
    clf.add(layer)
clf.add(Dense(2, activation='softmax'))
clf.summary()
clf.compile(optimizer='adam', loss='categorical_crossentropy')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 256)            264192    
                                                                 
 lstm_1 (LSTM)               (None, 4, 256)            525312    
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 1,315,330
Trainable params: 1,315,330
Non-trainable params: 0
_________________________________________________________________


In [ ]:
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})

In [ ]:
ip = df['review'][0]
la = df['sentiment'][0]

In [ ]:
def indexed_ip(tokens):
  indexed_ip =[]
  for token in tokens:
    if token in word_ind:
      indexed_ip.append(word_ind[token])
    else:
      indexed_ip.append(vocab_len)


In [ ]:
def prepareDataset():
  x_train = []
  y_train = []
  x_test =  []
  y_test =  []
  for i in range(5000):
    x=tokenizer.tokenize(preprocessString(df['review'][i]))
    x_train.append(indexed_ip(x))
    y_train.append(df['sentiment'][i])
  return x_train,y_train
  for i in range(500):
    x=tokenizer.tokenize(preprocessString(df['review'][i+5000]))
    x_test.append(indexed_ip(x))
    y_test.append(df['sentiment'][i+5000])
  return x_train,y_train,x_test,y_test

In [ ]:
x_train,y_train,x_test,y_test = prepareDataset()

ValueError: ignored

208154

'the'

In [ ]:
model.fit(X, Y, epochs=3, batch_size=32, callbacks=callbacks) 


Epoch 1/3
62/63 [============================>.] - ETA: 0s - loss: 6.2621
Epoch 00001: loss improved from 6.31763 to 6.26212, saving model to weights-improvement-01-6.2621.hdf5
63/63 [==============================] - 5s 83ms/step - loss: 6.2621
Epoch 2/3
63/63 [==============================] - ETA: 0s - loss: 6.2575
Epoch 00002: loss improved from 6.26212 to 6.25753, saving model to weights-improvement-02-6.2575.hdf5
63/63 [==============================] - 5s 85ms/step - loss: 6.2575
Epoch 3/3
62/63 [============================>.] - ETA: 0s - loss: 6.2389
Epoch 00003: loss improved from 6.25753 to 6.23287, saving model to weights-improvement-03-6.2329.hdf5
63/63 [==============================] - 8s 132ms/step - loss: 6.2329


Loading pretrained weights